In [11]:
# import necessary libraries
from bokeh.plotting import figure, show
from bokeh.embed import components
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.transform import  factor_cmap
import pandas as pd
from pymongo import MongoClient

### Importing data into MongoDB

* start mongodb server in terminal (file path is absolute file path):

    "C:\Program Files\MongoDB\Server\7.0\bin\mongod.exe" --dbpath="c:\data\db"

* navigate to folder containing 'emdat_cleaned.json' file, open new terminal, and run the following command:

    mongoimport --type json -d project3 -c disasters --drop --jsonArray emdat_cleaned.json

In [12]:
#initiate instance of pymongo
mongo = MongoClient(port=27017)

In [13]:
#assign database to python variable
db = mongo.project3

In [14]:
#Query the 'disasters' to confirm correct data import
db.disasters.find_one()

{'_id': ObjectId('6614478beb3befb6796ea102'),
 'Disaster #': '1999-9388',
 'Subgroup': 'Climatological',
 'Type': 'Drought',
 'Subtype': 'Drought',
 'Country': 'Djibouti',
 'Region': 'Africa',
 'Geolocation': 'Bugabira',
 'Magnitude': None,
 'Lat': -2.43007525,
 'Lng': 30.01978072,
 'Year': 2001,
 'Month': 6.0,
 'Deaths': None,
 'Injuries': None,
 'Total Affected': 100000.0,
 'Insured Damage (Adjusted)': None,
 'Total Damage (Adjusted)': None}

In [15]:
#query the whole collection and convert to a list of dictionaries
from_db = db.disasters.find({})
from_db = list(from_db)

In [16]:
#convert to a pandas dataframe for analysis
data = pd.DataFrame(from_db)
data.drop('_id', axis = 1, inplace=True)
data.head()

,Disaster #,Subgroup,Type,Subtype,Country,Region,Geolocation,Magnitude,Lat,Lng,Year,Month,Deaths,Injuries,Total Affected,Insured Damage (Adjusted),Total Damage (Adjusted)
0,1999-9388,Climatological,Drought,Drought,Djibouti,Africa,Bugabira,NaN,-2.430075,30.019781,2001,6.0,NaN,NaN,100000.0,NaN,NaN
1,1999-9388,Climatological,Drought,Drought,Djibouti,Africa,Buganda,NaN,-2.990570,29.225784,2001,6.0,NaN,NaN,100000.0,NaN,NaN
2,1999-9388,Climatological,Drought,Drought,Djibouti,Africa,Rugombo,NaN,-2.821939,29.094707,2001,6.0,NaN,NaN,100000.0,NaN,NaN
3,1999-9388,Climatological,Drought,Drought,Djibouti,Africa,Gihanga,NaN,-3.197334,29.294292,2001,6.0,NaN,NaN,100000.0,NaN,NaN
4,1999-9388,Climatological,Drought,Drought,Djibouti,Africa,Busoni,NaN,-2.473186,30.252261,2001,6.0,NaN,NaN,100000.0,NaN,NaN


In [17]:
#number of unique disaster id's in the dataset
data['Disaster #'].nunique()

5341

In [20]:
#Create df for disaster count by year
year_df = data[['Year', 'Type', 'Disaster #']].groupby(['Year', 'Type']).count()
year_df

Disaster #
Year Type                           
2000 Drought                     403
     Earthquake                   46
     Extreme temperature         248
     Flood                       564
     Mass movement (dry)           2
...                              ...
2018 Mass movement (wet)          21
     Storm                       632
     Volcanic activity            18
2019 Drought                       3
     Storm                         4

[145 rows x 1 columns]

In [21]:
# unstack df to get a more usable dataframe for graphing
year_df = year_df.unstack(level=1)
year_df

Disaster #                                                             \
Type    Drought Earthquake Extreme temperature   Flood Mass movement (dry)   
Year                                                                         
2000      403.0       46.0               248.0   564.0                 2.0   
2001      427.0       98.0               128.0   529.0                 NaN   
2002      222.0       70.0                75.0   584.0                 1.0   
2003      251.0       81.0              2418.0   929.0                 NaN   
2004       72.0     1035.0               544.0   875.0                 1.0   
2005      104.0      112.0              4644.0  1091.0                 NaN   
2006       45.0       61.0              3344.0  1352.0                 1.0   
2007       48.0       65.0              2089.0  1209.0                 NaN   
2008      114.0       51.0               110.0  1172.0                 4.0   
2009      802.0       87.0              1167.0   991.0                 2.0   
2010      400.0       47.0              1248.0  1225.0                 NaN   
2011      340.0      228.0               175.0  1439.0                 NaN   
2012      243.0       79.0              9761.0   604.0                 1.0   
2013       53.0       80.0               141.0   930.0                 1.0   
2014      162.0       66.0               188.0   726.0                 NaN   
2015      282.0      248.0                81.0  1145.0                 1.0   
2016      114.0       94.0               308.0  1037.0                 NaN   
2017       46.0       89.0               194.0  1194.0                 2.0   
2018       37.0       51.0                33.0   585.0                 1.0   
2019        3.0        NaN                 NaN     NaN                 NaN   

                                                    
Type Mass movement (wet)   Storm Volcanic activity  
Year                                                
2000                40.0   381.0               8.0  
2001                34.0   333.0              18.0  
2002                40.0   483.0              16.0  
2003                42.0   593.0               8.0  
2004                17.0  1619.0               8.0  
2005                14.0  1509.0              15.0  
2006                58.0   425.0              50.0  
2007                14.0   963.0              10.0  
2008                16.0  1133.0              32.0  
2009                38.0  1220.0               6.0  
2010                70.0  1147.0              19.0  
2011                27.0   633.0              24.0  
2012                17.0   911.0               3.0  
2013                14.0   813.0               5.0  
2014                26.0   702.0              33.0  
2015                66.0   953.0              20.0  
2016                46.0   994.0               NaN  
2017                57.0  3284.0               2.0  
2018                21.0   632.0              18.0  
2019                 NaN     4.0               NaN

In [23]:
#Create line plot of disasters over time. Output will be one graph with 8 lines (1 for each disaster type)

#Create chart format
count_by_year = figure(title="# of Disasters by Year", x_axis_label='Year', y_axis_label='# of Disasters', width = 1900, height = 500)

#generate each line of the chart
count_by_year.line(year_df.index, year_df.iloc[:,0], legend_label="Drought", color = '#ffee65')
count_by_year.line(year_df.index, year_df.iloc[:,1], legend_label="Earthquake", color = '#fdcce5')
count_by_year.line(year_df.index, year_df.iloc[:,2], legend_label="Extreme Temp.", color = '#fd7f6f')
count_by_year.line(year_df.index, year_df.iloc[:,3], legend_label="Flood", color = '#7eb0d5')
count_by_year.line(year_df.index, year_df.iloc[:,4], legend_label="Mass Mov. (Dry)", color = '#bd7ebe')
count_by_year.line(year_df.index, year_df.iloc[:,5], legend_label="Mass Mov. (Wet)", color = '#8bd3c7')
count_by_year.line(year_df.index, year_df.iloc[:,6], legend_label="Storm", color = '#b2e061')
count_by_year.line(year_df.index, year_df.iloc[:,7], legend_label="Volcanic Activity", color = '#ffb55a')

#save Javascript code and html code and show plot in browser
js_by_year, html_by_year = components(count_by_year)
show(count_by_year)

In [ ]:
#Print Javascript code for above plot to be copy and pasted into html or 'app.js'
#Optionally could be exported as a stand alone html file or .png
print(js_by_year)

#had to clear output before pushing to github to avoid errors displaying in github.

In [ ]:
#print html <div> to be placed in html file wherever it needs to show up on the webpage
print(html_by_year)

#had to clear output before pushing to github to avoid errors displaying in github.

In [27]:
#get subdataframe of the disaster count for each region
region_df = data[['Region', 'Type', 'Disaster #']].groupby(['Region', 'Type']).count()
region_df

Disaster #
Region   Type                           
Africa   Drought                    1536
         Earthquake                  378
         Extreme temperature          66
         Flood                      3689
         Mass movement (dry)           1
         Mass movement (wet)          59
         Storm                       765
         Volcanic activity            10
Americas Drought                    1107
         Earthquake                  318
         Extreme temperature        1290
         Flood                      4230
         Mass movement (dry)           4
         Mass movement (wet)         107
         Storm                      8512
         Volcanic activity           156
Asia     Drought                    1175
         Earthquake                 1861
         Extreme temperature        2019
         Flood                      6957
         Mass movement (dry)           8
         Mass movement (wet)         468
         Storm                      6951
         Volcanic activity           106
Europe   Drought                     291
         Earthquake                   87
         Extreme temperature       23511
         Flood                      2934
         Mass movement (dry)           2
         Mass movement (wet)          16
         Storm                      2040
         Volcanic activity             3
Oceania  Drought                      59
         Earthquake                   44
         Extreme temperature          10
         Flood                       371
         Mass movement (dry)           2
         Mass movement (wet)           7
         Storm                       464
         Volcanic activity            20

In [28]:
# unstack df to get a more usable dataframe for graphing
region_df = region_df.unstack(level=1)
region_df

Disaster #                                                           \
Type        Drought Earthquake Extreme temperature Flood Mass movement (dry)   
Region                                                                         
Africa         1536        378                  66  3689                   1   
Americas       1107        318                1290  4230                   4   
Asia           1175       1861                2019  6957                   8   
Europe          291         87               23511  2934                   2   
Oceania          59         44                  10   371                   2   

                                                      
Type     Mass movement (wet) Storm Volcanic activity  
Region                                                
Africa                    59   765                10  
Americas                 107  8512               156  
Asia                     468  6951               106  
Europe                    16  2040                 3  
Oceania                    7   464                20

In [29]:
#Create grouped barchart for comparing the count of disaster types per region

#Create list of factors for the barchart
x = [(x,y) for x  in region_df.index for y in region_df['Disaster #'].columns]

#define data sources for chart
sources = ColumnDataSource(data=dict(x = x, counts=region_df['Disaster #'].values))

#create chart dimensions and format
type_by_region = figure(x_range=FactorRange(*x), title=f"Disaster Count by Region", x_axis_label='Region (Disaster Type)', y_axis_label='Disaster Count',width=1500, height=400)

colors = ['#ffee65', '#fdcce5', '#fd7f6f', '#7eb0d5', '#bd7ebe', '#8bd3c7', '#b2e061', '#ffb55a']

#populate chart with data. Each bar color corresponds to a disaster type
type_by_region.vbar(x = 'x', top = 'counts', width=0.8, source = sources,
                    fill_color = factor_cmap('x', palette = colors, factors = region_df['Disaster #'].columns, start=1, end=2))

#Adjust x axis lables for readability
type_by_region.xaxis.major_label_orientation = 1.5

#save Javascript code and html code and show plot in browser
js_by_region, html_by_region = components(type_by_region)
#show(type_by_region)

In [ ]:
# print Javascript to be copy and pasted into 'app.js'
print(js_by_region)

#had to clear output before pushing to github to avoid errors displaying in github.

In [ ]:
#print html <div> to be placed in html file wherever it needs to show up on the webpage
print(html_by_region)

#had to clear output before pushing to github to avoid errors displaying in github.